<a href="https://colab.research.google.com/github/vesyapdw/Komputasi-Intelejensia-Tasks/blob/main/TaskWeek7_Vesya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama: Vesya Padmadewi

NPM: 2106726926

Model: https://huggingface.co/ayameRushia/roberta-base-indonesian-1.5G-sentiment-analysis-smsa

# Run the model

In [1]:
!pip install transformers

In [2]:
from transformers import pipeline

In [3]:
pipe = pipeline(
"text-classification",
model="ayameRushia/roberta-base-indonesian-1.5G-sentiment-analysis-smsa"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/990 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/504M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/822k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/468k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
pipe("Saya sangat senang hari ini")

[{'label': 'POSITIVE', 'score': 0.9997294545173645}]

In [5]:
pipe("Layanan sangat buruk sekali.")

[{'label': 'NEGATIVE', 'score': 0.9998482465744019}]

In [6]:
pipe("Saya pergi ke kantor.")

[{'label': 'NEUTRAL', 'score': 0.9995784163475037}]

In [7]:
pipe("Hari ini cuacanya cerah.")

[{'label': 'POSITIVE', 'score': 0.9995953440666199}]

In [8]:
pipe("Terima kasih atas bantuannya!")

[{'label': 'POSITIVE', 'score': 0.99969482421875}]

In [9]:
pipe("Ini sangat jauh dari ekspektasi.")

[{'label': 'NEGATIVE', 'score': 0.9995928406715393}]

In [10]:
pipe("Saya kecewa dengan pelayanannya.")

[{'label': 'NEGATIVE', 'score': 0.9998434782028198}]

In [11]:
# Create a synthetic dataset
synthetic_data = [
    {"text": "Terima kasih atas bantuannya ya!", "label": "positive"},
    {"text": "Layanan sangat buruk sekali.", "label": "negative"},
    {"text": "Pengalaman yang menyenangkan!", "label": "positive"},
    {"text": "Saya kecewa dengan produk ini.", "label": "negative"},
    {"text": "Produk ini luar biasa dan sangat membantu.", "label": "positive"},
    {"text": "Sungguh pengalaman belanja yang memuaskan!", "label": "positive"},
    {"text": "Produk ini cepat rusak dan tidak berguna.", "label": "negative"},
    {"text": "Ini adalah produk terbaru dari perusahaan kami.", "label": "neutral"},
    {"text": "Barang ini memiliki ukuran yang berbeda-beda.", "label": "neutral"},
    {"text": "Sistem sedang dalam pemeliharaan.", "label": "neutral"},
    {"text": "Jam buka toko adalah pukul 9 pagi.", "label": "neutral"},
    {"text": "Barang ini tidak sesuai dengan ekspektasi.", "label": "negative"},
]


# Evaluate model predictions
correct = 0
for item in synthetic_data:
    prediction = pipe(item["text"])[0]["label"]
    if prediction.lower() == item["label"]:
        correct += 1

accuracy = (correct / len(synthetic_data)) * 100
print(f"Model accuracy on synthetic data: {accuracy:.2f}%")

Model accuracy on synthetic data: 91.67%


In [12]:
# Trying synthetic data with just positive and negative datasets
synthetic_data2 = [
    {"text": "Terima kasih atas bantuannya ya!", "label": "positive"},
    {"text": "Layanan sangat buruk sekali.", "label": "negative"},
    {"text": "Pengalaman yang menyenangkan!", "label": "positive"},
    {"text": "Saya kecewa dengan produk ini.", "label": "negative"},
]


# Evaluate model predictions
correct = 0
for item in synthetic_data2:
    prediction = pipe(item["text"])[0]["label"]
    if prediction.lower() == item["label"]:
        correct += 1

accuracy = (correct / len(synthetic_data2)) * 100
print(f"Model accuracy on synthetic data: {accuracy:.2f}%")

Model accuracy on synthetic data: 100.00%


From this output, the decreased accuracy when neutral data is added in the synthetic dataset occurs because the model used is most likely only trained to recognize two sentiment categories: “positive” and ”negative.” **If the model only understands those two labels, any neutral sentence will still be predicted as positive or negative, resulting in a decrease in accuracy.**

# Attention transformers

In [21]:
!pip install transformers torch sklearn

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [22]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

model_name = "ayameRushia/roberta-base-indonesian-1.5G-sentiment-analysis-smsa"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [23]:
class AttentionLayer(nn.Module):
    def __init__(self, hidden_size):
        super(AttentionLayer, self).__init__()
        self.attention_weights = nn.Parameter(torch.rand(hidden_size))

    def forward(self, inputs):
        attention_scores = torch.tanh(inputs @ self.attention_weights)
        attention_weights = torch.softmax(attention_scores, dim=1).unsqueeze(-1)
        weighted_output = inputs * attention_weights
        return torch.sum(weighted_output, dim=1)

class CustomSentimentModel(nn.Module):
    def __init__(self, base_model):
        super(CustomSentimentModel, self).__init__()
        self.base_model = base_model
        self.attention = AttentionLayer(hidden_size=768)  # Match RoBERTa hidden size
        self.classifier = nn.Linear(768, 3)  # Output layer for 3 classes

    def forward(self, input_ids, attention_mask):
        outputs = self.base_model.roberta(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state  # (batch_size, seq_len, hidden_size)
        attention_output = self.attention(hidden_states)
        logits = self.classifier(attention_output)
        return logits

# Instantiate the custom model
model = CustomSentimentModel(base_model).to(device)

In [24]:
# Synthetic dataset for evaluation
synthetic_texts = [
    "Terima kasih atas bantuannya ya!", "Layanan sangat buruk sekali.",
    "Pengalaman yang menyenangkan!", "Saya kecewa dengan produk ini.",
    "Produk ini luar biasa dan sangat membantu.", "Sungguh pengalaman belanja yang memuaskan!",
    "Produk ini cepat rusak dan tidak berguna.", "Ini adalah produk terbaru dari perusahaan kami.",
    "Barang ini memiliki ukuran yang berbeda-beda.", "Sistem sedang dalam pemeliharaan.",
    "Jam buka toko adalah pukul 9 pagi.", "Barang ini tidak sesuai dengan ekspektasi."
]
synthetic_labels = [1, 0, 1, 0, 1, 1, 0, 2, 2, 2, 2, 0]  # 0=negative, 1=positive, 2=neutral

class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "labels": torch.tensor(label, dtype=torch.long)
        }

# Split into training and evaluation datasets
train_texts, eval_texts, train_labels, eval_labels = train_test_split(
    synthetic_texts, synthetic_labels, test_size=0.2, random_state=42
)
train_dataset = SentimentDataset(train_texts, train_labels, tokenizer)
eval_dataset = SentimentDataset(eval_texts, eval_labels, tokenizer)

In [25]:
optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

def train_model(model, train_dataset, eval_dataset, epochs=3, batch_size=8):
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    eval_loader = DataLoader(eval_dataset, batch_size=batch_size)

    for epoch in range(epochs):
        # Training phase
        model.train()
        total_loss = 0
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            # Forward pass
            logits = model(input_ids, attention_mask=attention_mask)
            loss = criterion(logits, labels)
            total_loss += loss.item()

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

        avg_train_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {avg_train_loss:.4f}")

        # Evaluation phase
        model.eval()
        correct_predictions = 0
        total_predictions = 0
        with torch.no_grad():
            for batch in eval_loader:
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["labels"].to(device)

                # Forward pass
                logits = model(input_ids, attention_mask=attention_mask)
                _, preds = torch.max(logits, dim=1)

                correct_predictions += torch.sum(preds == labels)
                total_predictions += labels.size(0)

        accuracy = correct_predictions.double() / total_predictions
        print(f"Epoch {epoch + 1}/{epochs}, Evaluation Accuracy: {accuracy:.4f}")

# Train the model
train_model(model, train_dataset, eval_dataset)

Epoch 1/3, Training Loss: 0.9731
Epoch 1/3, Evaluation Accuracy: 1.0000
Epoch 2/3, Training Loss: 0.2331
Epoch 2/3, Evaluation Accuracy: 1.0000
Epoch 3/3, Training Loss: 0.0778
Epoch 3/3, Evaluation Accuracy: 1.0000


In [26]:
def evaluate_model(model, texts, labels):
    dataset = SentimentDataset(texts, labels, tokenizer)
    data_loader = DataLoader(dataset, batch_size=8)
    model.eval()
    correct_predictions = 0
    total_predictions = 0
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            logits = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(logits, dim=1)

            correct_predictions += torch.sum(preds == labels)
            total_predictions += labels.size(0)

    accuracy = correct_predictions.double() / total_predictions
    return accuracy.item()

# Fine-tuned model accuracy
fine_tuned_accuracy = evaluate_model(model, synthetic_texts, synthetic_labels)
print(f"Fine-tuned Model Accuracy with Attention on synthetic data: {fine_tuned_accuracy * 100:.2f}%")

Fine-tuned Model Accuracy with Attention on synthetic data: 100.00%


After using attention transformers, the original model compared with the fine-tuned model that has implemented attention transformers, have the same level of accuracy: 100%.